In [158]:
# !pip install earthengine-api
# !pip install geehydro
# !pip install geemap
# !pip install geetools
# !pip install jupyter_contrib_nbextensions
# !pip install ndvi2gif

In [159]:
import ee
import folium
import geehydro
import geemap
import geetools
import os 

In [160]:
# ee.Authenticate()
ee.Initialize()

In [170]:
d = ee.ImageCollection('NOAA/PERSIANN-CDR').select('precipitation')
d = d.first().propertyNames()
print(d.getInfo())

['system:time_start', 'month', 'system:footprint', 'system:time_end', 'system:version', 'system:id', 'system:asset_size', 'system:index', 'system:bands', 'system:band_names']


In [151]:
Map = geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [152]:
landsat7 = ee.Image('LE7_TOA_5YEAR/1999_2003') \
    .select([0, 1, 2, 3, 4, 6])
landsat_vis = {
    'bands': ['B4', 'B3', 'B2'], 
    'gamma': 1.4
}
Map.addLayer(landsat7, landsat_vis, "LE7_TOA_5YEAR/1999_2003")

hyperion = ee.ImageCollection('EO1/HYPERION') \
    .filter(ee.Filter.date('2016-01-01', '2017-03-01'));
hyperion_vis = {
  'min': 1000.0,
  'max': 14000.0,
  'gamma': 2.5,
}
Map.addLayer(hyperion, hyperion_vis, 'EO1/HYPERION');

In [140]:
m = geemap.Map()
m.add_basemap('Google Satellite')
m

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [155]:
soums = ee.FeatureCollection('users/ta346/soum')

In [157]:
m.addLayer(soums, {}, 'soum')

In [80]:
from ndvi2gif import NdviSeasonality
myclass2 = NdviSeasonality(soums, 1985, 2020, 'Landsat', 'median')
median = myclass2.get_year_composite().median()
vizParams = {'bands': ['summer', 'autumn', 'winter'], 'min': 0, 'max': 0.7, 'gamma': [0.95, 1.1, 1]}
m.addLayer(median, vizParams, 'soum')

In [82]:
myclass2.get_gif()

Generating URL...
Please wait ...
The GIF image has been saved to: U:\honors thesis\gee_python\mygif.gif


# EE TimeSeries and TimeLapse

## Main functions

naip_timeseries(roi=None, start_year=2009, end_year=2018):

landsat_timeseries() #return collection from landsat

landsat_ts_gif() #Generates a Landsat timelapse GIF image.

add_text_to_gid() #Adds animated text to a GIF image.

add_image_to_gif() #Adds an image logo to a GIF image.

reduce_gif_size() #Reduces a GIF image using ffmpeg.

landsat_ts_norm_diff() #Computes a normalized difference index based on a Landsat timeseries.

landsat_ts_norm_diff_gif() #summary - uses landsat_ts_norm_diff()

Map.add_landsat_ts_gif() #add gif to the map and download the gif


In [6]:
Map = geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [143]:
roi = ee.Geometry.Polygon(
    [[[-115.471773, 35.892718],
      [-115.471773, 36.409454],
        [-114.271283, 36.409454],
        [-114.271283, 35.892718],
        [-115.471773, 35.892718]]], None, False)

In [144]:
collection = geemap.landsat_timeseries(roi=roi, start_year=1985, end_year=2019, start_date='06-10', end_date='09-20')

In [149]:
Map2 = geemap.Map()
Map2

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [150]:
first_image = collection.first()

vis = {
    'bands': ['NIR', 'Red', 'Green'],
    'min': 0,
    'max': 4000,
    'gamma': [1, 1, 1]
}

Map2.addLayer(first_image, vis, 'First image')

In [54]:
roi = Map.draw_last_feature
roi = roi.geometry()

In [124]:
colorizedVis = {
  'min': 0.0,
  'max': 1.0,
  'palette': [
    'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',
    '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',
    '012E01', '011D01', '011301'
  ],
}

In [125]:
Map.add_landsat_ts_gif(layer_name='TimeLapse',
                       roi=roi,
                       start_year=1985, 
                       end_year=2020, 
                       start_date='06-01', 
                       end_date='08-31', 
                       dimensions=768, 
                       frames_per_second=5, 
                       nd_bands=['NIR', 'Red'], 
                       nd_palette=colorizedVis)

Generating URL...
Please wait ...
An error occurred while downloading.
Generating URL...
Error in map(ID=2):
Image.visualize: Expected a string or list of strings for field 'palette'.
Adding animated text to GIF ...
The input gif file does not exist.
The input gif file does not exist.
The input gif file does not exist.
The input gif file does not exist.
Adding GIF to the map ...
The provided file does not exist.
The provided file does not exist.
The timelapse has been added to the map.


In [88]:
#mask cloud, snow, shadow ...for landsat 5-8
#Documentation: https://github.com/gee-community/gee_tools/blob/master/geetools/cloud_mask.py 
from geetools import cloud_mask
mask_LS_SR = cloud_mask.landsatSR()

In [128]:
#ndvi function for image collections for landsat 8
def ndviFun_ls8(im):
  ndvi = im.normalizedDifference(['B5','B4']).rename('ndvi').copyProperties(im, ['system:time_start'])
  return im.addBands(ndvi)

#ndvi function for landsat 5 and 7 image collections
def ndviFun_ls57(im):
  ndvi = im.normalizedDifference(['B4','B3']).rename('ndvi').copyProperties(im, ['system:time_start'])
  return im.addBands(ndvi)

#Sensor calibaration functions

#landsat 8 to 7 using coefficient found in D.P.Roy et el (2016)
def applyCoefficientL7(image):
  image_adjusted = image.select('ndvi').toFloat().copyProperties(image, ['system:time_start'])
  return(image_adjusted)

def applyCoefficientL8(image):
  image_adjusted = image.select('ndvi').multiply(0.9589).toFloat().copyProperties(image, ['system:time_start'])
  return(image_adjusted)

#landsat 5 to landsat 7 using coefficient found in Junchang and Masek (2017)
def applyCoefficientL5(image):
  image_adjusted = image.select('ndvi').multiply(1.037).toFloat().copyProperties(image, ['system:time_start'])
  return(image_adjusted)

In [129]:
# Read soum shapefiles from GEE assets
soums = ee.FeatureCollection('users/ta346/soum')
Map.addLayer(soums, {}, 'Soums')
Map.centerObject(soums)

In [130]:
#2020 LANDCOVER 30m
# //Source: http://www.globallandcover.com/defaults_en.html?src=/Scripts/map/defaults/En/download_en.html&head=download&type=data
# //name: 2020 global landcover 30m
# //downloaded locally and imported as an asset

# //Classification codes: Cultivated Land：10；Forest：20；Grass Land：30；Shrubland：40；Wetland：50；Water Body：60；Tundra：70；Artificial Surfaces：80；Bareland：90；Permanent Snow and Ice：100
# //mask: wetland 50, waterbody 60, Artificial surfaces 80, Permenant snow and ice 100
landcover = ee.Image('users/ta346/2020LC30')
lc = landcover.clip(soums).select('b1')

#to make landcover mask and display - masked areas are black (i.e zero)
mask_lc = lc.neq(50).And(lc.neq(60)).And(lc.neq(80)).And(lc.neq(100))
Map.addLayer(mask_lc, {}, 'landcover (30m)') #Looks Ok!

#function to map over the ndvi dataset collection
def maskLandCover (img):
    return img.updateMask(mask_lc).copyProperties(img, ['system:time_start'])

In [131]:
out_dir = r'U:\honors thesis\gee_python\downloads\gif'
out_gif = os.path.join(out_dir, "landsat_ts_2985_2019.gif")

In [135]:
# create collection images of landsat 5, 7, and 8; calibrated, filtered for 6, 7, 8 months.
l8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
    .filterBounds(soums) \
    .filter(ee.Filter.dayOfYear(120, 240)) \
    .filterDate('2013-05-01', '2020-08-31') \
    .map(mask_LS_SR) \
    .map(maskLandCover) \
    .map(ndviFun_ls8) \
    .map(applyCoefficientL8)
l7 = ee.ImageCollection('LANDSAT/LE07/C01/T1_SR') \
    .filterBounds(soums) \
    .filter(ee.Filter.dayOfYear(120, 240)) \
    .filterDate('1999-05-01', '2020-08-31') \
    .map(mask_LS_SR)  \
    .map(maskLandCover) \
    .map(ndviFun_ls57) \
    .map(applyCoefficientL7) 
l5 = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR') \
    .filterBounds(soums) \
    .filter(ee.Filter.dayOfYear(120, 240)) \
    .filterDate('1984-05-01', '2011-08-31') \
    .map(mask_LS_SR) \
    .map(maskLandCover) \
    .map(ndviFun_ls57) \
    .map(applyCoefficientL5)

# merge all the collections
mc = ee.ImageCollection(l8.merge(l7.merge(l5))).select('ndvi')

In [136]:
#Download ndvi the collection by distinct date and each soum - newcol 
#first, set the dates
start = ee.Date('2020-05-01')
finish = ee.Date('2020-08-31')

# Difference in days between start and finish
diff = finish.difference(start, 'day')
print(diff.getInfo())

temporalResolution = 7  # days - can be adjusted so that collect image collection by 8 days, 16 days, etc

# Make a list of all dates
def function(day):
    return start.advance(day,'day')

# date range
range_dates = ee.List.sequence(0, diff.subtract(1), temporalResolution).map(function)
print(range_dates.size().getInfo())

# for each distinct date in temoralresoultion, mosaic and return as a list of images
def day_mosaics(date, newlist):
  # Cast
  date = ee.Date(date)
  newlist = ee.List(newlist)

  # Filter collection between date and the next day
  filtered = mc.filterDate(date, date.advance(temporalResolution, 'day'))

  # Make the mosaic
  image = ee.Image(filtered.reduce(ee.Reducer.mean()).clip(soums)) \
                    .set("system:index", date.format("YYYMMdd")) 

  # Add the mosaic to a list only if the collection has images
  return ee.List(ee.Algorithms.If(filtered.size(), newlist.add(image), newlist))

#Create image collection from the list of images
newcol = ee.ImageCollection(ee.List(range_dates.iterate(day_mosaics, ee.List([]))))
print(newcol.size().getInfo())

122
18


EEException: User memory limit exceeded.

In [112]:
m2 = geemap.Map()
m2.addLayer(yearlyNDVI, colorizedVis, 'Summer Median')
m2

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

  Actual type: Float<-1.037, 1.037>.
     Image ID: null
This band might require an explicit cast.
DYCI3PX6II6V3I6LBVIBFPSS  Export.table  daily_ndvi_by_soum_2000                     FAILED     Expected a homogeneous image collection, but an image with an incompatible band was encountered. Mismatched type for band 'ndvi':
Expected type: Float<-1.0, 1.0>.
  Actual type: Float<-1.037, 1.037>.
     Image ID: null
This band might require an explicit cast.
DMLZZGQ2ZUWMKVZXLVXSMHQO  Export.table  daily_ndvi_by_soum_2000                     FAILED     Expected a homogeneous image collection, but an image with an incompatible band was encountered. Mismatched type for band 'ndvi':
Expected type: Float<-1.0, 1.0>.
  Actual type: Float<-1.037, 1.037>.
     Image ID: null
This band might require an explicit cast.
2XF575VC5EUZFCWZ6E6LQYM2  Export.table  daily_ndvi_by_soum_2014                     FAILED     Expected a homogeneous image collection, but an image with an incompatible band was encounte

HLEBDY55JEHL3LUGTVTZYLL7  Export.table  daily_ndvi_by_soum_2018                     CANCELLED  Cancelled.
3C2JYWQEOTPPD7BS2ZBO3DNL  Export.table  daily_ndvi_by_soum_2017                     CANCELLED  Cancelled.
F5QBEVZVVIEDURE6O7UM2DMT  Export.table  daily_ndvi_by_soum_2016                     CANCELLED  Cancelled.
O5OW3V3RDH5FOQBRTYWOGSMJ  Export.table  daily_ndvi_by_soum_2015                     CANCELLED  Cancelled.
XTBDJIENUCPBIUY5APUOG4GY  Export.table  daily_ndvi_by_soum_2014                     CANCELLED  Cancelled.
4W2YVR4L7G4GU5MXNEQFHU6H  Export.table  daily_ndvi_by_soum_2013                     CANCELLED  Cancelled.
W22MKEIF2B2EFM2V22FAYX2V  Export.table  daily_ndvi_by_soum_2012                     CANCELLED  Cancelled.
X7YXHDME3FAMTIV4ABT7TGHI  Export.table  daily_ndvi_by_soum_2011                     CANCELLED  Cancelled.
QZZ5APWWQUXVKHBQI5PYQ2L2  Export.table  daily_ndvi_by_soum_2010                     CANCELLED  Cancelled.
OYVAZA5VMAAM65A5VEWWRXLX  Export.table  daily_


5I6B3ZQDITRH4OJ57ZJBF22R  Export.table  daily_ndvi_by_soum_2000                     FAILED     ImageCollection.mosaic: Error in map(ID=LE07_122027_20000525):
Image.select: Pattern 'ndvi' did not match any bands.
EWNR2RAB7RGH6RE3EIG5PTX5  Export.table  daily_ndvi_by_soum_2000                     FAILED     ImageCollection.mosaic: Error in map(ID=LE07_122027_20000509):
Image.select: Pattern 'ndvi' did not match any bands.
VDZYHARSQQH5GUCQSWNY52ZO  Export.table  daily_ndvi_by_soum_2000                     FAILED     Expected a homogeneous image collection, but an image with an incompatible band was encountered. Mismatched type for band 'ndvi':
Expected type: Float<-1.0, 1.0>.
  Actual type: Float<-1.037, 1.037>.
     Image ID: null
This band might require an explicit cast.
LZKXE4GR53LIRUH3MESQAQPB  Export.table  daily_ndvi_by_soum_2000                     FAILED     Expected a homogeneous image collection, but an image with an incompatible band was encountered. Mismatched type for band '

IO7IVUMUJNFV5JOIQ5UJHWJH  Export.table  daily_ndvi_by_soum_2008                     CANCELLED  Cancelled.
6XM73QN3EJT7AROJ5KM3AOOY  Export.table  daily_ndvi_by_soum_2007                     CANCELLED  Cancelled.
Q23GQGZXHQ7O66HPKPZW5Z4M  Export.table  daily_ndvi_by_soum_2006                     CANCELLED  Cancelled.
GYOTY6XV7IE3WMNNTXW7U2R4  Export.table  daily_ndvi_by_soum_2005                     CANCELLED  Cancelled.
NFDZ64TKLWZFFI6RX6BILV5O  Export.table  daily_ndvi_by_soum_2004                     CANCELLED  Cancelled.
JQHZAUFREMRQFZJPB4M3PFC7  Export.table  daily_ndvi_by_soum_2003                     CANCELLED  Cancelled.
XVM4ZT3NLTNDP6DHKSLIMFMC  Export.table  daily_ndvi_by_soum_2002                     CANCELLED  Cancelled.
ZW4JIPNOVJ52GFCUQQFAY4AY  Export.table  daily_ndvi_by_soum_2001                     CANCELLED  Cancelled.
VRR5GYQZJNIMYXEFNMBFEQ3T  Export.table  daily_ndvi_by_soum_2000                     CANCELLED  Cancelled.
W2NZGCKUQ6RYSEEJEKMQNYVO  Export.table  daily_

In [139]:
# !earthengine task list